In [ ]:

from huggingface_hub import hf_hub_download

# Arc2Face 모델
hf_hub_download("FoivosPar/Arc2Face", "arc2face/config.json",           local_dir="models/arc2face")
hf_hub_download("FoivosPar/Arc2Face", "arc2face/diffusion_pytorch_model.safetensors", local_dir="models/arc2face")
hf_hub_download("FoivosPar/Arc2Face", "encoder/config.json",             local_dir="models/encoder")
hf_hub_download("FoivosPar/Arc2Face", "encoder/pytorch_model.bin",       local_dir="models/encoder")

# AntelopeV2 + ArcFace ONNX
!mkdir -p models/antelopev2
# (zip 해제 or hf_hub_download로 arcface.onnx까지 models/antelopev2에 위치)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Colab에서 실행
!mkdir -p models/antelopev2
!unzip "/content/drive/MyDrive/antelopev2.zip" -d models/antelopev2

In [ ]:
hf_hub_download(repo_id="FoivosPar/Arc2Face", filename="arcface.onnx", local_dir="./models/antelopev2")

In [ ]:
%pip install --upgrade \
  "huggingface_hub==0.17.3" \
  "transformers==4.41.0" \
  "diffusers==0.23.0" \
  "accelerate==0.20.3"


In [ ]:
from diffusers import (
    StableDiffusionPipeline,
    UNet2DConditionModel,
    DPMSolverMultistepScheduler,
)
import sys, os
# Arc2Face 클론 경로에 맞게 변경
sys.path.append(os.path.abspath("/content/Arc2Face"))


from arc2face import CLIPTextModelWrapper, project_face_embs

import torch
from insightface.app import FaceAnalysis
from PIL import Image
import numpy as np

# Arc2Face is built upon SD1.5
# The repo below can be used instead of the now deprecated 'runwayml/stable-diffusion-v1-5'
base_model = 'stable-diffusion-v1-5/stable-diffusion-v1-5'

encoder = CLIPTextModelWrapper.from_pretrained(
    'models', subfolder="encoder", torch_dtype=torch.float16
)

unet = UNet2DConditionModel.from_pretrained(
    'models', subfolder="arc2face", torch_dtype=torch.float16
)

pipeline = StableDiffusionPipeline.from_pretrained(
        base_model,
        text_encoder=encoder,
        unet=unet,
        torch_dtype=torch.float16,
        safety_checker=None
    )

In [ ]:
from diffusers import StableDiffusionPipeline, UNet2DConditionModel, DPMSolverMultistepScheduler
from arc2face import CLIPTextModelWrapper
import torch

# encoder, unet 로드
encoder = CLIPTextModelWrapper.from_pretrained('models', subfolder="encoder", torch_dtype=torch.float16)
unet    = UNet2DConditionModel.from_pretrained('models', subfolder="arc2face", torch_dtype=torch.float16)

# 파이프라인 생성
pipeline = StableDiffusionPipeline.from_pretrained(
    'stable-diffusion-v1-5',
    text_encoder=encoder,
    unet=unet,
    torch_dtype=torch.float16,
    safety_checker=None
)

# 스케줄러 교체 및 CUDA 이동
pipeline.scheduler = DPMSolverMultistepScheduler.from_config(pipeline.scheduler.config)
pipeline = pipeline.to('cuda')


In [ ]:
# 1) 맞춤 버전 설치: cached_download 도 있고 split_* 도 있는 버전
%pip install huggingface_hub==0.25.2

# 2) (선택) diffusers, transformers, accelerate 버전 고정
%pip install --upgrade diffusers==0.23.0 transformers==4.34.1 accelerate==0.20.3


In [ ]:
import torch
from diffusers import StableDiffusionPipeline, UNet2DConditionModel, DPMSolverMultistepScheduler
from arc2face import CLIPTextModelWrapper, project_face_embs

# 1) 텍스트 인코더와 U-Net 로드
encoder = CLIPTextModelWrapper.from_pretrained(
    "models", subfolder="encoder", torch_dtype=torch.float16
)
unet = UNet2DConditionModel.from_pretrained(
    "models", subfolder="arc2face", torch_dtype=torch.float16
)

# 2) Stable Diffusion 기반 Arc2Face 파이프라인 생성
pipeline = StableDiffusionPipeline.from_pretrained(
    "stable-diffusion-v1-5",
    text_encoder=encoder,
    unet=unet,
    torch_dtype=torch.float16,
    safety_checker=None,
)

# 3) 스케줄러 교체 및 CUDA로 이동
pipeline.scheduler = DPMSolverMultistepScheduler.from_config(pipeline.scheduler.config)
pipeline = pipeline.to("cuda")


In [ ]:
from insightface.app import FaceAnalysis
from PIL import Image
import numpy as np

# 1) AntelopeV2 얼굴 분석 객체 준비
app = FaceAnalysis(name="antelopev2", root=".", providers=["CUDAExecutionProvider","CPUExecutionProvider"])
app.prepare(ctx_id=0, det_size=(640,640))

# 2) 예시 이미지 로드 -> 가장 큰 얼굴 선택
img = np.array(Image.open("assets/examples/joacquin.png"))[:,:,::-1]
faces = app.get(img)
# bbox 면적 기준 가장 큰 face
face = sorted(faces, key=lambda x:(x["bbox"][2]-x["bbox"][0])*(x["bbox"][3]-x["bbox"][1]))[-1]

# 3) ArcFace 임베딩 생성 및 정규화
id_emb = torch.tensor(face["embedding"], dtype=torch.float16)[None].cuda()
id_emb = id_emb / torch.norm(id_emb, dim=1, keepdim=True)

# 4) Arc2Face 텍스트 인코더를 통해 ID 토큰 자리에 임베딩 주입
prompt_embeds = project_face_embs(pipeline, id_emb)


In [ ]:
# 원하는 생성 파라미터
num_images = 4
steps       = 25
guidance    = 3.0

# 1) 생성
outputs = pipeline(
    prompt_embeds=prompt_embeds,
    num_inference_steps=steps,
    guidance_scale=guidance,
    num_images_per_prompt=num_images
)

# 2) PIL.Image 리스트로 결과 획득
images = outputs.images  # length == num_images

# 3) 결과 저장 및 표시
for i, im in enumerate(images):
    fn = f"/content/output_id{i}.png"
    im.save(fn)
    display(im)      # Colab 환경에서 이미지 바로 보기
    print("saved to", fn)
